![Image](actinia_logo.png)

## Sentinel-2 NDVI computation

Actinia provides several API calls to compute satellite specific
parameters:

 <https://actinia.mundialis.de/api_docs/#tag-Satellite-Image-Algorithms>

This tutorial shows the actinia REST service access using the Python
library **requests**.

The following example is a jupyter notebook version of the online [actinia tutorial](https://actinia-dev.mundialis.de/tutorial/tutorial_sentinel2a_ndvi.html). 

---

### actinia API documentation

* [Stable actinia API v3 docs](https://redocly.github.io/redoc/?url=https://actinia.mundialis.de/api/v3/swagger.json)
* [Development actinia API v3 docs](https://redocly.github.io/redoc/?url=https://actinia-dev.mundialis.de/api/v3/swagger.json)

---

### Requirements

#### Software & Modules

This tutorial assumes your are comfortable with the [Python](https://python.org) programming language. Familiarity with basic REST API concepts and usage is also assumed.

Python modules used in this tutorial are:
* [requests](http://docs.python-requests.org/)
* [json](https://docs.python.org/3/library/json.html)


#### ACTINIA API user and password

This demo requires credentials for authentication set below in **Preparation** as a variable. Another actinia instance might require different credentials.

### Helper Modules and Functions
Before interacting with the actinia server using Python, we will import required packages an set up a helper function to print formatted JSON using json.

***Note:*** *You may need to install two helpful browser plugins called **RESTman** and **JSON Formatter** that format JSON and makes it easier to read:*

* [RESTman extension](https://chrome.google.com/webstore/detail/restman/ihgpcfpkpmdcghlnaofdmjkoemnlijdi)
* [JSON Formatter](https://chrome.google.com/webstore/detail/json-formatter/bcjindcccaagfpapjjmafapmmgkkhgoa)

## Preparation


In [ ]:
# first, let's import the required packages.

from pprint import pprint
import sys
import json
import time

import requests
from requests.auth import HTTPBasicAuth


To simplify our life in terms of server communication we store the credentials and REST server URL in  variables.

In [ ]:
# variables to set the actinia host, version, and user

actinia_baseurl = "https://actinia.mundialis.de"
actinia_version = "v3"
actinia_url = actinia_baseurl + "/api/" + actinia_version
actinia_auth = HTTPBasicAuth('demouser', 'gu3st!pa55w0rd')

In [ ]:
# helper function to print formatted JSON using the json module

def print_as_json(data):
    print(json.dumps(data, indent=2))

# helper function to verify a request
def verify_request(request, success_code=200):
    if request.status_code != success_code:
        print("ERROR: actinia processing failed with status code %d!" % request.status_code)
        print("See errors below:")
        print_as_json(request.json())
        request_url = request.json()["urls"]["status"]
        requests.delete(url=request_url, auth=actinia_auth)
        raise Exception("The resource <%s> has been terminated." % request_url)

## Example

The following asynchronous API call computes the NDVI of the Sentinel-2
scene
**S2A_MSIL1C_20161206T030112_N0204_R032_T50RKR_20161206T030749**:

In [ ]:
# make a POST request to the actinia data API
request_url = actinia_url + "/sentinel2_process/ndvi/S2A_MSIL1C_20161206T030112_N0204_R032_T50RKR_20161206T030749"
print("actinia POST request:")
print(request_url)
print("---")
request = requests.post(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

The response provides the status URL that must be polled to receive the
finished response:

In [ ]:
# get a json-encoded content of the response
jsonResponse = request.json()

print("Response with status code %d:" % request.status_code)

# print formatted JSON
print_as_json(jsonResponse)

Poll the status of the asynchronous API call by polling the status URL.
Be aware that you have to use the current status url as the resource id will change for different NDVI API
calls.

In [ ]:
# make a GET request to the actinia data API
request_url = jsonResponse["urls"]["status"]
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

The final result will contain a complete processing list as well as
URL's to the resulting PNG preview image, the created NDVI GeoTiff file
as well as univariate statistics of the computed NDVI scene.

Continue polling until there is a message **"Processing successfully finished"**

In [ ]:
# continue polling until finished
while request.status_code == 200 and \
        jsonResponse["message"] != "Processing successfully finished":
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

# check if anything went wrong
verify_request(request, 200)

Full response:

In [ ]:
# print formatted JSON
print_as_json(jsonResponse)

The statistical analysis of the resulting NDVI raster map layer is
available in the *process_results* section:

In [ ]:
# print formatted JSON
print_as_json(jsonResponse["process_results"])

The links in the urls section of the JSON response point to the resulting PNG preview image and the
NDVI GeoTiff file:

In [ ]:
# print formatted JSON
print_as_json(jsonResponse["urls"]["resources"])